In [9]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# URL del archivo en el bucket público de AWS
url = "https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
len(df)

# ¡¡¡6,362,620 OBSERVACIONES!!!

In [ ]:
## 1.1. Inspección de datos (solo para verificación)
print("Información del DataFrame:")
df.info()

# Contar valores nulos (opcional, pero buena práctica)
print("\nValores nulos por columna:")
print(df.isnull().sum())
# Generalmente, este dataset no tiene nulos.

## 1.2. Separación de Características (X) y Etiqueta (y)
# La columna 'Class' es la variable objetivo
X = df.drop('Class', axis=1)
y = df['Class']

## 1.3. Escalado de Características (Time y Amount)
# El StandardScaler transforma los datos para que tengan una media de 0 y una desviación estándar de 1.
scaler = StandardScaler()

# Ajustar y transformar las columnas 'Time' y 'Amount'
X['Time'] = scaler.fit_transform(X[['Time']])
X['Amount'] = scaler.fit_transform(X[['Amount']])

print("\nPrimeras filas de X después del escalado:")
print(X.head())

In [ ]:
## 2.1. División del Conjunto de Datos
# Usamos un 80% para entrenamiento y un 20% para prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")

## 2.2. Entrenamiento del Modelo de Regresión Logística
# Inicializar el modelo
model = LogisticRegression(solver='liblinear', random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

print("\n¡El modelo de Regresión Logística ha sido entrenado exitosamente!")

In [ ]:
## 3.1. Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

## 3.2. Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(conf_matrix)

# Interpretación de la Matriz de Confusión:
# [[ Verdaderos Negativos (TN) | Falsos Positivos (FP) ]
#  [ Falsos Negativos (FN)   | Verdaderos Positivos (TP) ]]

## 3.3. Reporte de Clasificación Final
print("\nReporte de Clasificación:")
# Este reporte es la evaluación final que debes capturar.
reporte_final = classification_report(y_test, y_pred)
print(reporte_final)